# Предсказание результатов матча в игре Dota 2 по событиям первых 5 минут игры

## Подготовка данных
Загрузим таблицу с признаками из файла features.csv и удалим признаки, связанные с итогами матча.
Поместим в X набор признаков, в y - столбец с целевой переменной radiant_win(победа команды radiant)

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import time
import datetime

data = pd.read_csv('features.csv', index_col='match_id')
X = data.loc[:, 'start_time':'dire_first_ward_time']
y = data['radiant_win']

Проверим, есть ли у нас признаки с пропусками значений, посчитаем количество пропусков

In [2]:
nan_count = X.isnull().sum()
print(nan_count[nan_count != 0])

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64


Пропуски в датасете связаны с особенностью выборки, ведь мы анализируем только первые 5 минут матча. В первые 5 минут могут не произойти некоторые события(первое убийство героя - first blood, покупка предмета bottle командой radiant - radiant_bottle_time, покупка предмета ward командой dire - dire_first_ward_time)

Заменим все пропущенные значения в этом датасете на нули:

In [3]:
X.fillna(inplace=True, value=0)

Теперь попробуем обучить два разных классификатора и сравним их качество предсказаний

## Подход 1 - градиентный бустинг
Разделим обучающую выборку на 5 блоков с помощью генератора KFold и обучим наш классификатор для различного количества деревьев. Будем считать качество классификатора по метрике ROC_AUC и время, в течение которого настраивалась модель

In [10]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

last_score = 0.5  # для сравнения качество случайного распределения

for trees in range(10, 110, 10):
    start_time = datetime.datetime.now()
    gbm_score = []
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
    
        gbm = GradientBoostingClassifier(n_estimators=trees, subsample=1.0, max_depth=3,
                                         random_state=241)
                
        gbm.fit(X_train, y_train)
        pred = gbm.predict_proba(X_test)[:, 1]
        gbm_score.append(roc_auc_score(y_test, pred))
        
    time = (datetime.datetime.now() - start_time).total_seconds()
    score = np.mean(gbm_score)
    print(f'{trees:3d} деревьев | качество: {score:5.3f} | время обучения: ',
          f'{time:5.1f} c | прирост качества в сек: {(score-last_score)/time:7.5f}')
        
    last_score = score

 10 деревьев | качество: 0.665 | время обучения:   18.7 c | прирост качества в сек: 0.00884
 20 деревьев | качество: 0.682 | время обучения:   34.2 c | прирост качества в сек: 0.00051
 30 деревьев | качество: 0.690 | время обучения:   50.4 c | прирост качества в сек: 0.00015
 40 деревьев | качество: 0.694 | время обучения:   65.8 c | прирост качества в сек: 0.00006
 50 деревьев | качество: 0.697 | время обучения:   82.4 c | прирост качества в сек: 0.00004
 60 деревьев | качество: 0.700 | время обучения:   98.5 c | прирост качества в сек: 0.00003
 70 деревьев | качество: 0.702 | время обучения:  114.2 c | прирост качества в сек: 0.00002
 80 деревьев | качество: 0.704 | время обучения:  130.5 c | прирост качества в сек: 0.00001
 90 деревьев | качество: 0.705 | время обучения:  148.5 c | прирост качества в сек: 0.00001
100 деревьев | качество: 0.706 | время обучения:  165.4 c | прирост качества в сек: 0.00001


При увеличении количества деревьев увеличивается качество классификации и время обучения. Видим, что оптимальное количество деревьев - 30, при дальнейшем увеличении количества деревьев непропорционально возрастают затраты времени.
Чтобы ускоить обучение при увеличении количества деревьев, можно либо уменьшать размеры подвыборки(параметр subsample), либо уменьшить максимальную глубину дерева(параметр max_depth)

## Подход 2 - логистическая регрессия

Для логистической регрессии важно выполнить предварительное масштабирование признаков с помощью StandartScaler

In [55]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype(float))
print(X_scaled)

[[-2.54436416  1.54068827 -1.24422828 ... -0.55115386  1.84600409
  -1.10802864]
 [-2.54045236 -0.92779756 -0.29225805 ...  0.67817009  0.43778816
   0.04672892]
 [-2.53923104  1.54068827 -0.5686365  ...  0.67817009  0.43778816
   0.4889765 ]
 ...
 [ 1.09874571 -0.57515673  1.42743012 ...  0.67817009  0.43778816
  -0.19896418]
 [ 1.09895204 -0.57515673  1.48884755 ...  0.67817009  0.43778816
  -0.86233554]
 [ 1.1026479   1.54068827 -0.04658831 ... -0.55115386 -0.97042777
  -0.78862761]]


Подберём коэффициент C для регрессии


Настройка параметра регуляризации


Теперь найдем оптимальное (в данном примере) значение параметра регуляризации . Сделать это можно с помощью LogisticRegressionCV – перебора параметров по сетке с последующей кросс-валидацией. Этот класс создан специально для логистической регрессии (для нее известны эффективные алгоритмы перебора параметров), для произвольной модели мы бы использовали GridSearchCV, RandomizedSearchCV или, например, специальные алгоритмы оптимизации гиперпараметров, реализованные в hyperopt.


Код

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

c_values = np.logspace(-2, 3, 500)

logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, verbose=1, n_jobs=-1)
logit_searcher.fit(X_poly, y)

Посмотрим, как качество модели (доля правильных ответов на обучающей и валидационной выборках) меняется при изменении гиперпараметра .



Обучим модель 

In [ ]:
logreg = LogisticRegression(penalty='l2', )

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?
3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).
4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?
6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).